# 01. Data Preparation

1. Load the source training set and source training labels.
2. Add columns that will be needed for generating the collections of event sequences per question.
3. Perform basic EDA.

In [1]:
import pandas as pd

## Load Source Data

In [12]:
# load the source training set
df_source = pd.read_csv('data/train.csv.gz', compression='gzip', index_col=1)

print(df_source.shape)
with pd.option_context('display.max_columns', None):
    display(df_source.head(3))

(13174211, 19)


,session_id,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
index,,,,,,,,,,,,,,,,,,,
0,20090312431273200,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [13]:
# load the source training labels
df_source_labels = pd.read_csv('data/train_labels.csv')

print(df_source_labels.shape)
with pd.option_context('display.max_columns', None):
    display(df_source_labels.head(3))

(212022, 2)


,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090314121766812_q1,1


## Handle Duplicate Rows

In [22]:
# show the number of duplicates
print('Number of duplicates in the source training set: {} ({})'.format(df_source.duplicated().sum(), df_source.duplicated().sum() / df_source.shape[0]))
print('Number of duplicates in the source training labels: {}'.format(df_source_labels.index.duplicated().sum()))

Number of duplicates in the source training set: 766 (5.814389947147499e-05)
Number of duplicates in the source training labels: 0


In [24]:
# remove duplicate rows and reset the index
df_unique = df_source \
    .drop_duplicates() \
    .reset_index(drop=True)

print(df_unique.shape)

(13173445, 19)


## Prepare the Label Dataset

Additional columns need to be added to the labeling data set to allow matching to a collection of events 